# About
* **Author**: Adil Rashitov
* **Created at**: 07.08.2021
* **Goal**: Solve final assignment

![pictures](pictures/FINAL_TASK_1.png)

In [1]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Multiprocessing for Mac / Linux
import platform
platform.system()
if platform.system() == 'Darwin':
    from multiprocess import Pool
else:
    from multiprocessing import Pool

# Visualization libraries
import plotly.express as px

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

from ortools.linear_solver import pywraplp

# Data

In [2]:
# Data reading & prepatre
def prepare_data():

    DATA = """
    1	7	None
    2	4	5, 8
    3	6	None
    4	9	None
    5	12	2, 8
    6	8	9
    7	10	10
    8	11	2, 5
    9	8	6
    10	7	7
    11	6	15
    12	8	None
    13	15	None
    14	14	None
    15	3	11
    """


    DATA = DATA.split('\n')[1:-1]
    DATA = list(map(lambda x: x.split('\t'), DATA))
    data = pd.DataFrame(DATA, columns=['Job', 'Processing time', 'Conflicting jobs'])
    data['Job'] = data['Job'].str.lstrip().str.rstrip()
    data['Processing time'] = data['Processing time'].astype(int)
    return data


In [3]:
jobs_df = prepare_data()
horizon = int(jobs_df['Processing time'].sum())


N_MACHINES = 3
N_JOBS = jobs_df.shape[0]
logging.info(f"{N_JOBS} jobs need to assigned to {N_MACHINES} machines")
logging.info(f"Jobs dataframe:")
jobs_df
logging.info(f"Horizon: {horizon}")

[ 09/10/2021 11:38:43 AM ][ INFO ]: 15 jobs need to assigned to 3 machines
[ 09/10/2021 11:38:43 AM ][ INFO ]: 15 jobs need to assigned to 3 machines
[ 09/10/2021 11:38:43 AM ][ INFO ]: Jobs dataframe:
[ 09/10/2021 11:38:43 AM ][ INFO ]: Jobs dataframe:


,Job,Processing time,Conflicting jobs
0,1,7,None
1,2,4,"5, 8"
2,3,6,None
3,4,9,None
4,5,12,"2, 8"
5,6,8,9
6,7,10,10
7,8,11,"2, 5"
8,9,8,6
9,10,7,7


[ 09/10/2021 11:38:43 AM ][ INFO ]: Horizon: 128


,Job,Processing time,Conflicting jobs
0,1,7,None
1,2,4,"5, 8"
2,3,6,None
3,4,9,None
4,5,12,"2, 8"
5,6,8,9
6,7,10,10
7,8,11,"2, 5"
8,9,8,6
9,10,7,7


[ 09/10/2021 11:38:43 AM ][ INFO ]: Horizon: 128


# Main


### Step 1 : Model definition

In [4]:
# Model.
from ortools.sat.python import cp_model
model = cp_model.CpModel()


# Change variable
X  = np.array([[model.NewBoolVar(f"JOB_{job}_MACHINE_{machine}") for job in range(N_JOBS)] for machine in range(N_MACHINES)])

# Cost vector
J = np.array(jobs_df['Processing time'])

### Step 2 : Objective function formulation

Answer to problem with definiton of objective function lies: https://groups.google.com/g/or-tools-discuss/c/BJhIMaVQ2w8/m/yfMcPamUFgAJ


> AddMaxEquality() expects an array of variables, not an array of expressions. Please create intermediate variables for each .all_tasks[job_id, len(job) - 1].deadline - all_tasks[job_id, len(job) - 1].end.




Laurent Perron | Operations Research | lpe...@google.com | (33) 1 42 68 53 00


In [5]:
# Objective function

# 1. Creation of intermediate list holding make span for each machine
makespans = []
for machine_id, makespan in enumerate(X @ J):
    makespans.append(model.NewIntVar(0, horizon, f'machine_span_{machine_id}'))
    model.Add(makespans[-1] == makespan)

# 2. Overall span minimization
overall_span = model.NewIntVar(0, horizon, 'overall_span')
model.AddMaxEquality(overall_span, makespans)
model.Minimize(overall_span)

### Step 3 : Constrains add

---

* **Modeling constrains**:
    * **Constrain 1**: Each job must be assigned only once
* **Input constrains**:
    * **Constrain 2**: Predefined constrains

**Constrain 1**: Each job must be assigned only once

In [6]:
[model.Add(x_job == 1) for x_job in sum(X)]

### 4. Solving optimization

In [7]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [8]:
solver.ObjectiveValue()

43.0

43.0

### 5. Results visualization

In [10]:
X_computed = [[solver.Value(X[row][col]) for col in range(X.shape[1])] for row in range(X.shape[0])]


[[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]]

[[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]]